In [2]:
import numpy as np # linear algebra
import pandas as pd
import os

In [3]:
import pandas as pd
train=pd.read_excel('Data_Train.xlsx')
test=pd.read_excel('Data_Test.xlsx')

In [4]:
train["source"] = "train"
test["source"] = "test"

In [5]:
df = pd.concat([train,test])

C:\Users\Shubham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4230
Data columns (total 10 columns):
CITY             16774 non-null object
COST             12690 non-null float64
CUISINES         16921 non-null object
LOCALITY         16793 non-null object
RATING           16917 non-null object
RESTAURANT_ID    16921 non-null int64
TIME             16921 non-null object
TITLE            16921 non-null object
VOTES            15315 non-null object
source           16921 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 1.4+ MB


In [7]:
df.duplicated().sum()

26

In [8]:
df=df.drop_duplicates()

In [9]:
df.isna().sum()

CITY              147
COST             4230
CUISINES            0
LOCALITY          128
RATING              4
RESTAURANT_ID       0
TIME                0
TITLE               0
VOTES            1602
source              0
dtype: int64

In [10]:
# Data exploration for CITY
# CITY has 147 null values
#combining City and locality
df['Location']=df['CITY']+' '+df['LOCALITY']
df.drop(columns=['CITY','LOCALITY'])

,COST,CUISINES,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,source,Location
0,1200.0,"Malwani, Goan, North Indian",3.6,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",CASUAL DINING,49 votes,train,Thane Dombivali East
1,1500.0,"Asian, Modern Indian, Japanese",4.2,13198,6pm – 11pm (Mon-Sun),"CASUAL DINING,BAR",30 votes,train,Chennai Ramapuram
2,800.0,"North Indian, Chinese, Biryani, Hyderabadi",3.8,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",CASUAL DINING,221 votes,train,Chennai Saligramam
3,800.0,"Tibetan, Chinese",4.1,6346,11:30am – 1am (Mon-Sun),QUICK BITES,24 votes,train,Mumbai Bandra West
4,300.0,Desserts,3.8,15387,11am – 1am (Mon-Sun),DESSERT PARLOR,165 votes,train,Mumbai Lower Parel
...,...,...,...,...,...,...,...,...,...
4226,NaN,"North Indian, Mughlai, Chinese",3.9,9057,11:30am – 11:30pm (Mon-Sun),CASUAL DINING,287 votes,test,New Delhi Punjabi Bagh
4227,NaN,"Biryani, North Indian, Sandwich, Salad, Wraps",4.3,1247,11am – 1am (Mon-Sun),None,469 votes,test,Bangalore HSR Layout
4228,NaN,"Continental, North Indian",3.7,8617,9:30am – 10:30pm (Mon-Sun),QUICK BITES,53 votes,test,Faridabad Sector 86
4229,NaN,"Rolls, Beverages",-,6485,"11am – 11:30pm (Mon, Tue, Wed, Thu, Sat, Sun),...",QUICK BITES,NaN,test,Kochi Kochi


In [11]:
df.dropna(subset=['Location'],inplace=True)

In [12]:
!pip install fuzzywuzzy

In [13]:
from fuzzywuzzy import process
 
names_array=[]
def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        if x[1]<60:
            names_array.append('Others')
        else:
            names_array.append(x[0])
    return names_array
  
#Wrong country names dataset

correct_names=['Bangalore','Thane',
'Hyderabad','Andheri',
'Delhi', 'Kerala',
'Chennai', 'Bandra',
'Mumbai', 'Telangana',
'Kochi', 
'Noida', 
'Gurgaon', 'Ernakulam',
'Faridabad', 'Ghaziabad',
'Secunderabad' ]
name_match=match_names(df.Location,correct_names)    

print(len(names_array))
df['Location']=names_array


C:\Users\Shubham\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


16747


In [14]:
cuisines_list=[]
for row in df['CUISINES']:
    cuisines_list.append(list(row.split(',')))

df['CUISINES']=cuisines_list

In [15]:
df['CUISINES'].isna().sum()

0

In [16]:
df_cuisines=df['CUISINES'].apply(lambda x: pd.Series(1, x))

In [17]:
title_list=[]
for row in df['TITLE']:
    title_list.append(list(row.split(',')))
df['TITLE']=title_list

In [18]:
df_title=df['TITLE'].apply(lambda x: pd.Series(1, x))

In [19]:
df_title.head()

,CASUAL DINING,BAR,QUICK BITES,DESSERT PARLOR,CAFÉ,MICROBREWERY,BEVERAGE SHOP,IRANI CAFE,BAKERY,None,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# cleaning time - pending

In [20]:
df[df['RATING'].isna()]

,CITY,COST,CUISINES,LOCALITY,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,source,Location
204,Kochi,200.0,[Bakery],Marine Drive,NaN,15062,Not Available,[BAKERY],NaN,train,Kochi
421,Hyderabad,350.0,"[Cafe, Fast Food]",Banjara Hills,NaN,14127,Not Available,[CAFÉ],NaN,train,Hyderabad
2035,Kochi,NaN,"[Kerala, Arabian]",Kakkanad,NaN,2776,Not Available,[QUICK BITES],NaN,test,Kochi
2758,Kochi,NaN,"[Chinese, North Indian, South Indian]",Tripunithura,NaN,11301,Not Available,[None],NaN,test,Kochi


In [21]:
df["RATING"] = df.groupby("CITY").RATING.transform(lambda x : x.fillna(x.mode()[0]))

In [22]:
df['RATING']=df['RATING'].str.extract('(\d+)').astype(float)

In [23]:
df['VOTES'].isna().sum()

1579

In [24]:
df.VOTES.fillna('0',inplace=True)
df['VOTES']=df['VOTES'].str.extract('(\d+)').astype(float)

In [25]:
df.drop(columns='CITY',inplace=True)
df.drop(columns='LOCALITY',inplace=True)
df.drop(columns='CUISINES',inplace=True)

In [26]:
#df.drop(columns='CUISINES',inplace=True)
#df.drop(columns='CUISINES++',inplace=True)
#df.drop(columns='Location++',inplace=True)
#df.drop(columns='TITLE++',inplace=True)


In [27]:
df_City=pd.get_dummies(df['Location'])
df.drop(columns='Location',inplace=True)
df_City.head()


,Andheri,Bandra,Bangalore,Chennai,Delhi,Ernakulam,Faridabad,Ghaziabad,Gurgaon,Hyderabad,Kerala,Kochi,Mumbai,Noida,Others,Secunderabad,Telangana,Thane
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [28]:
df = pd.concat([df,df_City,df_cuisines,df_title], axis=1)


In [29]:
df.drop(columns='TITLE',inplace=True)

In [30]:
df_column_category = df.select_dtypes(exclude=np.number).columns
df_column_category

Index(['TIME', 'source'], dtype='object')

In [31]:
#df.drop(columns='City found',inplace=True)
df.drop(columns='TIME',inplace=True)

In [32]:
df.fillna(0,inplace=True)

In [33]:
train_final = df[df.source=="train"]
test_final = df[df.source=="test"]

In [34]:
train_final.shape

(12552, 277)

In [35]:
train_final.drop(columns=["source"],inplace=True)

C:\Users\Shubham\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [36]:
test_final.drop(columns=["source",'COST'],inplace=True)


In [37]:
train_X = train_final.drop(columns=["COST",'RESTAURANT_ID'])

In [38]:
train_Y = train_final["COST"]

In [39]:
test_X = test_final.drop(columns=["RESTAURANT_ID"])

In [40]:
train_X.fillna(0,inplace=True)
train_X.isna().sum()

C:\Users\Shubham\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


RATING          0
VOTES           0
Andheri         0
Bandra          0
Bangalore       0
               ..
DHABA           0
MEAT SHOP       0
COCKTAIL BAR    0
PAAN SHOP       0
BHOJANALYA      0
Length: 274, dtype: int64

In [41]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train_X, train_Y)
dtrain_predictions = model.predict(train_X)

In [42]:
from sklearn.model_selection import cross_val_score
a = cross_val_score(model, train_X, train_Y, cv=5, scoring='neg_mean_squared_error')

In [43]:
#Print model report:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error, r2_score
print("\nModel Report")
print("RMSE : %.4g" % np.sqrt(mean_squared_error(train_Y.values, dtrain_predictions)))
    
#Predict on testing data:
test_X.fillna(0,inplace=True)
test_final["res_linear"] =  model.predict(test_X)



Model Report
RMSE : 331.3


C:\Users\Shubham\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs
C:\Users\Shubham\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
print('r2 train',r2_score(train_Y,dtrain_predictions))
#print('r2 test',r2_score(test_y,test_predict))

r2 train 0.7224620545102787


In [45]:
Linear_submission = test_final[["RESTAURANT_ID","res_linear"]]

In [46]:
Linear_submission.head(20)

,RESTAURANT_ID,res_linear
0,4085,917.578613
1,12680,359.179779
2,1411,809.393555
3,204,363.644165
4,13453,205.617432
5,4518,51.631500
6,1643,615.144043
7,5109,620.427124
8,5606,433.676636
9,14319,654.869659


In [48]:
from sklearn.externals import joblib
joblib.dump(model,"Restaurant_Food.pkl")

['Restaurant_Food.pkl']